https://pycrawfordprogproj.readthedocs.io/en/latest/Project_05/Project_05.html

We now try with this thing.

In [11]:
import time
import numpy as np
import pyscf

def restricted_HF_convE_and_DM(mol, conv_tol = 1e-10):
    """Restricted Hartree-Fock method, following Szabo and Ostlund.
    """
    start = time.time()

    n_a = mol.nelec[0]

    T = mol.intor("int1e_kin")
    Vnuc = mol.intor("int1e_nuc")
    hcore = T + Vnuc
    S = mol.intor("int1e_ovlp")
    h2e = mol.intor('int2e', aosym="s1") # most expensive, least intelligent way of doing this.

    convE = 10
    convDM = 10
    convComm = 10

    # we now implement checking convergence of both density matrix as well as energy.

    s, U = np.linalg.eigh(S)
    
    s_inv_sqrt = np.diag([x**(-0.5) for x in s])

    X = U @ s_inv_sqrt @ U.conj().T

    F0 = X.T @ hcore @ X
    _, C0 = np.linalg.eigh(F0)
    C = X @ C0
    P = 2 * C[:, :n_a] @ C[:, :n_a].T

    Eprev = None

    while convE > conv_tol or convDM > conv_tol or convComm > conv_tol:
        
        G = np.einsum('ij,klji->kl',P,h2e) - 0.5 * np.einsum('ij,kijl->kl',P,h2e)
        F = hcore + G

        E0 = 0.5 * np.einsum('ij,ij->',P,hcore+F)

        if Eprev is not None:
            convE = np.abs(E0 - Eprev)

        convComm = np.linalg.norm(F @ P @ S - S @ P @ F, 'fro')

        if convE < conv_tol and convDM < conv_tol and convComm < conv_tol:
            break
        
        Eprev = E0

        Fp = X.conj().T @ F @ X
        eps, Cp = np.linalg.eigh(Fp)
        C = X @ Cp

        Pnew = 2 * C[:,0:n_a] @ C[:,0:n_a].conj().T
        convDM = np.linalg.norm(Pnew-P)
        P = Pnew


        # if convE < conv_tol and convDM < conv_tol and convComm < conv_tol:

        #     Eprev = E0

        #     Fp = X.conj().T @ F @ X
        #     eps, Cp = np.linalg.eigh(Fp)
        #     C = X @ Cp

        #     Pnew = 2 * C[:,0:n_a] @ C[:,0:n_a].conj().T
        #     convDM = np.linalg.norm(Pnew-P)
        #     P = Pnew

        #     break
        
        # Eprev = E0

        # Fp = X.conj().T @ F @ X
        # eps, Cp = np.linalg.eigh(Fp)
        # C = X @ Cp

        # Pnew = 2 * C[:,0:n_a] @ C[:,0:n_a].conj().T
        # convDM = np.linalg.norm(Pnew-P)
        # P = Pnew

    E0 = E0 + mol.energy_nuc()

    end = time.time()
    print(f"Converged in {end - start} seconds")

    return E0, P, F, C


In [12]:
## pyscf calculation

from pyscf import gto, scf, cc
from pyscf.cc.ccd import CCD

import numpy as np
import scipy as sp

mol = gto.Mole()
mol.atom = """
H    0.0    0.0    0.0
O    1.0    0.0    0.0
H    2.0    0.0    0.0
"""
mol.basis = 'sto-3g'
mol.charge = 0
mol.spin = 0
mol.build()

print("orbitals:",mol.nao)
print("elec:",mol.nelec)

soln = restricted_HF_convE_and_DM(mol)

mf = scf.RHF(mol)
start = time.time()
mf.kernel()
end = time.time()

print(start - end, "seconds for RHF")


start = time.time()
mycc = cc.CCSD(mf)
mycc.kernel()
end = time.time()
print(start - end, "seconds for CCSD")

orbitals: 7
elec: (5, 5)
Converged in 0.26843905448913574 seconds
converged SCF energy = -74.8415921601715
-0.7769393920898438 seconds for RHF
E(CCSD) = -74.88203711153518  E_corr = -0.04044495136374247
-3.709019422531128 seconds for CCSD


In [9]:
import numpy as np
guessm = np.zeros((5, 5, 5))
guessm[1,2,3] = 1
guessm[1,3,4] = 2
# print(guessm)

guessmt = np.einsum('ijk->ikj',guessm)
# print(guessmt)

guessmt[0,0,0] = 3
print(guessm)

[[[3. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 2.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]]


In [ ]:
def ccsd(mol, solution, norb, nelec):

    nocc = nelec // 2
    nvirt = norb - nocc

    T = mol.intor("int1e_kin")
    Vnuc = mol.intor("int1e_nuc")
    hcore = T + Vnuc
    
    t2 = np.zeros((nocc,nocc,nvirt,nvirt)) # like ijab
    t1 = np.zeros((nocc,nvirt))

    h2e = mol.intor('int2e', aosym="s1") # most expensive, least intelligent way of doing this.

    mo_coeff = solution[3]
    fock = solution[2]
    fock_mo = mo_coeff.T @ fock @ mo_coeff

    mo_energy = np.diag(mo_coeff.T @ fock @ mo_coeff)

    D_ov = mo_energy[:nocc,None] - mo_energy[None,nocc:]

    # Occupied energies (εi, εj)
    ei = mo_energy[:nocc]      # shape (nocc,)
    ej = mo_energy[:nocc]      # shape (nocc,)

    # Virtual energies (εa, εb)
    ea = mo_energy[nocc:]      # shape (nvir,)
    eb = mo_energy[nocc:]      # shape (nvir,)

    # Compute εi + εj - εa - εb using broadcasting
    # Reshape for broadcasting:
    #   εi + εj: shape (nocc, nocc, 1, 1)
    #   εa + εb: shape (1, 1, nvir, nvir)
    D_oovv = ei[:, None, None, None] + ej[None, :, None, None] - ea[None, None, :, None] - eb[None, None, None, :]

    # Doovv[i,j,a,b] = ei + ej - ea - eb


    h1e_mo = mo_coeff.T @ hcore @ mo_coeff
    h2e_mo = np.einsum('ijkl,ip,jq,kr,ls->pqrs',h2e,mo_coeff,mo_coeff,mo_coeff,mo_coeff)

    v = h2e_mo

    w = 2*h2e_mo - np.transpose(h2e_mo,(0,3,2,1))

    so, sv = slice(0,nocc), slice(nocc, norb)

    # v and w are such that v^{ij}_{ab} has dims (i,a,j,b)

    w_ovoo = w[so,sv,so,so]
    w_ovvo = w[so,sv,sv,so]
    w_ovov = w[so,sv,so,sv]
    w_ovvv = w[so,sv,sv,sv]
    w_voov = w[sv,so,so,sv]
    w_vvov = w[sv,sv,so,sv]
    w_ooov = w[so,so,so,sv]
    v_oooo = v[so,so,so,so]
    v_ovoo = v[so,sv,so,so]
    v_ovvo = v[so,sv,sv,so]
    v_ovov = v[so,sv,so,sv]
    v_oovv = v[so,so,sv,sv]
    v_ovvv = v[so,sv,sv,sv]
    v_vvvv = v[sv,sv,sv,sv]
    v_voov = v[sv,so,so,sv]
    v_vvoo = v[sv,sv,so,so]
    v_vovv = v[sv,so,sv,sv]
    v_vooo = v[sv,so,so,so]

    # F^k_i and L^k_i has dims (k,i)

    converged = False

    iterations = 0

    while not converged:

        Foo = fock_mo[so,so] + np.einsum('kcld,ilcd->ki',w_ovov,t2) + np.einsum('kcld,ic,ld->ki',w_ovov,t1,t1)
        Fvv = fock_mo[sv,sv] - np.einsum('kcld,klad->ac',w_ovov,t2) - np.einsum('kcld,ka,ld->ac',w_ovov,t1,t1)
        Fov = np.einsum('kcld,ld->kc',w_ovov,t1)
        # Fvo = np.reshape(Fov,(1,0)) # i'm hoping this works?

        Loo = Foo + np.einsum('lcki,lc->ki',w_ovoo,t1)
        Lvv = Fvv + np.einsum('kdac,kd->ac',w_ovvv,t1)


        # W^{kl}_{ij} is stored for some godforsaken reason as (k,l,i,j).

        # transpose (0,1,2,3) in v --> (0,2,1,3)

        W_oooo = np.transpose(v_oooo,(0,2,1,3)) + np.einsum('lcki,jc->klij',v_ovoo,t1) + np.einsum('kclj,ic->klij',v_ovoo,t1) + np.einsum('kcld,ijcd->klij',v_ovov,t2) + np.einsum('kcld,ic,jd->klij',v_ovov,t1,t1)
        W_vvvv = np.transpose(v_vvvv,(0,2,1,3)) - np.einsum('kdac,kb->abcd',v_ovvv,t1) - np.einsum('kcbd,ka->abcd',v_ovvv,t1)
        W_voov = np.transpose(v_voov,(0,2,1,3)) - np.einsum('kcli,la->akic',v_ovoo,t1) + np.einsum('kcad,id->akic',v_ovvv,t1) - 0.5*np.einsum('ldkc,ilda->akic',v_ovov,t2) - np.einsum('ldkc,id,la->akic',v_ovov,t1,t1) + 0.5*np.einsum('ldkc,ilad->akic',w_ovov,t2)
        W_vovo = np.transpose(v_vvoo,(0,2,1,3)) - np.einsum('lcki,la->akci',v_ovoo,t1) + np.einsum('kdac,id->akci',v_ovvv,t1) - 0.5*np.einsum('lckd,ilda->akci',v_ovov,t2) - np.einsum('lckd,id,la->akci',v_ovov,t1,t1)

        Fvvt = Fvv - np.diag(mo_energy[nocc:])
        Foot = Foo - np.diag(mo_energy[:nocc])

        Loot = Loo - np.diag(mo_energy[:nocc])
        Lvvt = Lvv - np.diag(mo_energy[nocc:])

        rhs1 = np.einsum('ac,ic->ia',Fvvt,t1) - np.einsum('ki,ka->ia',Foot,t1) + np.einsum('kc,kica->ia',Fov,2*t2 - np.transpose(t2,(1,0,2,3))) + np.einsum('kc,ic,ka->ia',Fov,t1,t1) + np.einsum('aikc,kc->ia',w_voov,t1) + np.einsum('ackd,ikcd->ia',w_vvov,t2) + np.einsum('ackd,ic,kd->ia',w_vvov,t1,t1) - np.einsum('kilc,klac->ia',w_ooov,t2) - np.einsum('kilc,ka,lc->ia',w_ooov,t1,t1)

        t1 = rhs1 / D_ov

        rhs2_line1 = 0.5*np.einsum('iajb->ijab',v_ovov) + 0.5*np.einsum('klij,klab->ijab',W_oooo,t2) + 0.5*np.einsum('klij,ka,lb->ijab',W_oooo,t1,t1) + 0.5*np.einsum('abcd,ijcd->ijab',W_vvvv,t2) + 0.5*np.einsum('abcd,ic,jd->ijab',W_vvvv,t1,t1)
        rhs2_line23 = np.einsum('ac,ijcb->ijab',Lvvt,t2) - np.einsum('ki,kjab->ijab',Loot,t2) + np.einsum('aibc,jc->ijab',v_vovv,t1) - np.einsum('kibc,ka,jc->ijab',v_oovv,t1,t1) - np.einsum('aikj,kb->ijab',v_vooo,t1) - np.einsum('aikc,jc,kb->ijab',v_voov,t1,t1)
        rhs2_line4 = 2*np.einsum('akic,kjcb->ijab',W_voov,t2) - np.einsum('akci,kjcb->ijab',W_vovo,t2) - np.einsum('akic,kjbc->ijab',W_voov,t2) - np.einsum("bkci,kjac->ijab",W_vovo,t2)

        rhs2 = rhs2_line1 + rhs2_line23 + rhs2_line4 + np.copy(np.einsum('ijab->jiba',rhs2_line1 + rhs2_line23 + rhs2_line4))

        t2 = rhs2 / D_oovv

        Eccsd = np.einsum('iajb,ijab->',w_ovov,t2) + np.einsum('iajb,ia,jb->',w_ovov,t1,t1)

        print(Eccsd)

        iterations += 1

        if iterations == 100:
            converged = True

    return Eccsd, t1, t2  
        
    

In [ ]:
Eval, t1mat, t2mat = ccsd(mol, soln, 7, 10)

# ok so this somehow works but idfk why

-0.029330495948905664
-0.03692619339700498
-0.0392387618664338
-0.0400066424449909
-0.040277988496042784
-0.04037877690296536
-0.04041782615221254
-0.040433512477478985
-0.04044001141198723
-0.04044277475048382
-0.04044397509376797
-0.040444505566647256
-0.04044474322681868
-0.04044485084698241
-0.04044489998676953
-0.040444922568455215
-0.04044493299719302
-0.040444937832013014
-0.04044494008025227
-0.04044494112822762
-0.04044494161767803
-0.040444941846643954
-0.04044494195390263
-0.040444942004208354
-0.04044494202782793
-0.04044494203892884
-0.04044494204415099
-0.04044494204660982
-0.04044494204776857
-0.0404449420483151
-0.040444942048573095
-0.04044494204869499
-0.040444942048752625
-0.040444942048779894
-0.040444942048792835
-0.040444942048798956
-0.04044494204880187
-0.040444942048803244
-0.04044494204880388
-0.0404449420488042
-0.04044494204880437
-0.040444942048804416
-0.04044494204880446
-0.04044494204880447
-0.040444942048804486
-0.040444942048804486
-0.0404449420488045
-